In [33]:
from surface_dynamics.all import Origami
import numpy as np

In [34]:
def cycle_notation(array): 
    '''
        Argument: array of integers representing gluing instruction of origami, every element in array is glued to the next
        Return: string representing the cycle of the gluing
        Comment: assumes array is a permutation we are using which is a cycle as we are gluing the boxes
            until the edges meet, and is therefore NOT a general method for converting permutation to cycle notation, 
            it basically makes the array a string
    '''
    cycle = '('
    for i in array:
        cycle = cycle + str(int(i)) + ','
    cycle = cycle[:-1] + ')'
    return cycle

In [35]:
def enum_strata(n):
    '''
        Argument: integer n, size of origamis
        Return: array of origamis of the form two sheared rectangles stacked and aligned on left, 
            with bottom rectangle wider than top rectangle, they are in H(2)
        Comment: https://jamboard.google.com/d/1OSRaahlM2KiY1_onnQ5y6TDBOrseFGrHhtssKQc2ee0/edit?usp=sharing
            on slide 3 explains external gluing formula
    '''
    surfaces = []
    for u2 in range(2, n): # base rectangle has width [2, n-1)
        for h2 in range(1, round(n/u2) + 1): # base rectangle has height [1, n/u2)
            upperbound = min(u2, n-u2*h2 + 1) # top rectangle has width less than u2 or at most area of the top rectangle
            for u1 in range(1, upperbound): # top rectangle has width [1, upperbound)
                if (n-u2*h2)%u1 == 0: # as long as u1 divides the area, it's valid
                    h1 = (n-u2*h2)/u1
                    rectangle2 = np.arange(1, u2*h2 + 1)
                    rect2 = np.reshape(rectangle2, (h2, u2)) # base rectangle rect2
                    rect2 = np.flip(rect2,0)
                    rectangle1 = np.arange(u2*h2 + 1, n + 1)
                    rect1 = np.reshape(rectangle1, (h1, u1)) # top rectangle rect1
                    rect1 = np.flip(rect1,0)
                    hperm = '' 
                    # horizontal (right) permutation is independent of twist
                    for i in range(h1):
                        hperm += cycle_notation(rect1[i])
                    for i in range(h2):
                        hperm += cycle_notation(rect2[i])
                    for t1 in range(u1):
                        for t2 in range(u2):
                            marker = [1] * u2
                            vperm = ''
                            while sum(marker) != 0:
                                nextn = np.nonzero(marker)[0][0]
                                column = []
                                while marker[nextn] != 0:
                                    column = np.concatenate(([row[nextn] for row in rect2],column))
                                    marker[nextn] = 0
                                    if nextn < u1:
                                        col = [row[nextn] for row in rect1]
                                        column = np.concatenate((col, column))
                                        nextn = (u2-t2+(u1+nextn-t1)%u1)%u2
                                    else:
                                        nextn = (nextn+u2-t2)%u2
                                vperm += cycle_notation(np.flip(column,0))
                            origami = Origami(vperm, hperm)
                            surfaces.append(origami)
    return surfaces

In [36]:
def G(n):
    '''
        Argument: number n for input
        Return: the value of G(n) as in the paper
        Comment: only works for n up to 48 because the totient functions are hard coded
        https://arxiv.org/pdf/1810.08687.pdf
    '''
    Jordan_totient = [0, 1, 3, 8, 12, 24, 24, 48, 48, 72, 72, 120, 96, 168, 144, 192, 192, 288, 216, 360, 288, 384, 360, 528, 384, 600, 504, 648, 576, 840, 576, 960, 768, 960, 864, 1152, 864, 1368, 1080, 1344, 1152, 1680, 1152, 1848, 1440, 1728, 1584, 2208, 1536]
    Euler_totient = [0, 1, 1, 2, 2, 4, 2, 6, 4, 6, 4, 10, 4, 12, 6, 8, 8, 16, 6, 18, 8, 12, 10, 22, 8, 20, 12, 18, 12, 28, 8, 30, 16, 20, 16, 24, 12, 36, 18, 24, 16, 40, 12, 42, 20, 24, 22, 46, 16, 42, 20, 32, 24, 52, 18, 40, 24, 36, 28, 58, 16, 60, 30, 36, 32]
    g = 5/24*n*Jordan_totient[n]+1/2*n*Euler_totient[n]-3/4*Jordan_totient[n]
    return g

In [37]:
# This is a brief test for counting
for i in [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]:
    print(G(i) == len(enum_strata(i)))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
